In [1]:
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

%matplotlib inline


In [2]:
def interactive_lattice_plot(G,K):
    '''
    :param G,K: coupling
    '''
    k = np.linspace(-np.pi/2,np.pi/2,100)

    omega_o = G+K-np.sqrt((G+K)**2-4*G*K*np.sin(k)**2)
    omega_a = G+K+np.sqrt((G+K)**2-4*G*K*np.sin(k)**2)

    plt.plot(k,np.sqrt(omega_a))
    plt.plot(k,np.sqrt(omega_o))
    plt.xlabel("k")
    plt.ylabel("omega")
    plt.show()


In [3]:
interactive_plot = interactive(interactive_lattice_plot, G=(1, 6.0), K=(1, 6.0))
interactive_plot


interactive(children=(FloatSlider(value=3.5, description='G', max=6.0, min=1.0), FloatSlider(value=3.5, description='K', max=6.0, min=1.0), Output()), _dom_classes=('widget-interact',))